In [ ]:
import torch
from diff3f import get_features_per_vertex
from time import time
from utils import convert_mesh_container_to_torch_mesh, cosine_similarity, double_plot, get_colors, generate_colors
from dataloaders.mesh_container import MeshContainer
from diffusion import init_pipe
from dino import init_dino
from functional_map import compute_surface_map

In [2]:
device = torch.device('cuda:0')
torch.cuda.set_device(device)
num_views = 50
H = 512
W = 512
num_images_per_prompt = 1
tolerance = 0.004
random_seed = 42
use_normal_map = True

In [3]:
def compute_features(device, pipe, dino_model, m, prompt):
    mesh = convert_mesh_container_to_torch_mesh(m, device=device, is_tosca=False)
    mesh_vertices = mesh.verts_list()[0]
    features = get_features_per_vertex(
        device=device,
        pipe=pipe, 
        dino_model=dino_model,
        mesh=mesh,
        prompt=prompt,
        mesh_vertices=mesh_vertices,
        num_views=num_views,
        H=H,
        W=W,
        tolerance=tolerance,
        num_images_per_prompt=num_images_per_prompt,
        use_normal_map=use_normal_map,
    )
    return features.cpu()

In [ ]:
pipe = init_pipe(device)
dino_model = init_dino(device)

In [5]:
source_file_path = "meshes/cow.obj"
target_file_path = "meshes/camel.obj"

source_file_path = "shrec20_dataset/SHREC20b_lores/models/cow.obj"
target_file_path = "shrec20_dataset/SHREC20b_lores/models/camel_a.obj"

source_mesh = MeshContainer().load_from_file(source_file_path)
target_mesh = MeshContainer().load_from_file(target_file_path)

In [ ]:
f_source = compute_features(device, pipe, dino_model, source_mesh, "cow")

In [ ]:
f_target = compute_features(device, pipe, dino_model, target_mesh, "camel")

In [8]:
import numpy as np
s = cosine_similarity(f_source.to(device),f_target.to(device))
s = torch.argmax(s, dim=0).cpu().numpy()
np.save('predicted_mapping.npy', s)
cmap_source = get_colors(source_mesh.vert); cmap_target = cmap_source[s]

In [ ]:
double_plot(source_mesh,target_mesh,cmap_source,cmap_target)  

In [1]:
from eval import evaluate_meshes

# Call the evaluation function
avg_error, accuracy, distances = evaluate_meshes(
    source_file_path = "shrec20_dataset/SHREC20b_lores/models/cow.obj",
    target_file_path = "shrec20_dataset/SHREC20b_lores/models/camel_a.obj",
    source_gt_path = 'shrec20_dataset/SHREC20b_lores_gts/cow.mat',
    target_gt_path = 'shrec20_dataset/SHREC20b_lores_gts/camel_a.mat',
    mapping_path = 'predicted_mapping.npy', 
    debug=False
)

print(f"Average correspondence error (err): {avg_error:.6f}")
print(f"Correspondence accuracy (acc, γ=1%): {accuracy:.6f}")

Average correspondence error (err): 0.138106
Correspondence accuracy (acc, γ=1%): 0.729167


# Apply functional map on features

In [ ]:
source_file_path = "meshes/cat.off"
target_file_path = "meshes/lion.off"
source_mesh = MeshContainer().load_from_file(source_file_path)
target_mesh = MeshContainer().load_from_file(target_file_path)
f_source = compute_features(device, pipe, dino_model, source_mesh, "cat")
f_target = compute_features(device, pipe, dino_model, target_mesh, "lion")

In [ ]:
s = cosine_similarity(f_source.to(device),f_target.to(device))
s = torch.argmax(s, dim=0).cpu().numpy()
cmap_source = get_colors(source_mesh.vert); cmap_target = cmap_source[s]
double_plot(source_mesh,target_mesh,cmap_source,cmap_target)  

In [ ]:
surface_map = compute_surface_map(source_file_path, target_file_path, f_source.numpy(), f_target.numpy())

In [ ]:
cmap_source = get_colors(source_mesh.vert); cmap_target = cmap_source[surface_map.cpu().numpy()]
double_plot(source_mesh,target_mesh,cmap_source,cmap_target)  

# Part segmentation

In [29]:
from sklearn.cluster import KMeans
import numpy as np

k = 6

kmeans = KMeans(n_clusters=k, random_state=0, n_init="auto").fit(f_source)

segments1 = kmeans.predict(f_source)

# Apply centroids on another mesh to segment it in a corresponding manner
segments2 = kmeans.predict(f_target)

In [30]:
segment_colors = generate_colors(k)
cmap_source = np.array([segment_colors[j] for j in segments1])
cmap_target = np.array([segment_colors[j] for j in segments2])

In [ ]:
double_plot(source_mesh,target_mesh,cmap_source,cmap_target)  

In [ ]:
source_file_path = "meshes/posed_human.off"
target_file_path = "meshes/cat.off"
source_mesh = MeshContainer().load_from_file(source_file_path)
target_mesh = MeshContainer().load_from_file(target_file_path)
f_source = compute_features(device, pipe, dino_model, source_mesh, "naked human")
f_target = compute_features(device, pipe, dino_model, target_mesh, "cat")

In [40]:
k = 6

kmeans = KMeans(n_clusters=k, random_state=0, n_init="auto").fit(f_source)

segments1 = kmeans.predict(f_source)

# Apply centroids on another mesh to segment it in a corresponding manner

segments2 = kmeans.predict(f_target)
segment_colors = generate_colors(k)
cmap_source = np.array([segment_colors[j] for j in segments1])
cmap_target = np.array([segment_colors[j] for j in segments2])

In [ ]:
double_plot(source_mesh,target_mesh,cmap_source,cmap_target)  